In [ ]:
# HTML output for this notebook can be produced using
# jupyter nbconvert --to html --no-input notebook_name.ipynb
# or
# jupyter nbconvert --to pdf --no-input notebook_name.ipynb
# edit title and authors in notebook metadata (e.g. jupyter lab / notebook tools / advanced)

# Mahone Bay

See [TODO.md](../TODO.md) for outstanding items.

This notebook contains the same functionality as `tidal_analysis_rt.ipynb`, but with all detail code extracted into the `../acoustic_tracking` package code folder.  For this notebook to run properly, ensure:
* `acoustic_tracking` is downloaded and installed on your local machine according to the installation instructions
* The `acoustic_env` conda environment has been activated

Analysis of acoustic tracking performance using range test data obtained in Mahone Bay near Halifax, NS, by OTN field experiments during March-April 2016.

Range test performance is determined with two methods:

  1. by calculating interval lenghts between adjacent detection events,
  1. by counting the number of detection events within some fixed time interval and normalizing against the expected number of detections.

Components of this notebook:

 * process tidal data for the time period considering high/low tide times and the observed heights
 * determine tidal phase timing
 * perform cosine interpolation of heights
 * correlate detection performance against tidal phase

**Data:** Beyond tidal data, environmental variables have been collected for 3 hour intervals. Water velocity is used from those variables to determine its potential effect on detection performance.

Summary plots are presented separately for each receiver / transmitter combination at the end of this notebook.

In [ ]:
import range_driver as rd
import copy
import pandas as pd
import kadlu
from datetime import datetime
import numpy as np
import sys
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from matplotlib.pyplot import rcParams

# Setup matplotlib inline plotting
%matplotlib inline
rd.mpl_set_notebook_params()
show_details = False

# Set the kadlu storage location to point to cached data
kadlu.storage_cfg(setdir=f'{rd.utils.repo_path()}/data/MahoneBay2016/kadlu_data/')

In [ ]:
#rd.reload_acoustic()

In [ ]:
baseconfig = rd.yload(rd.load_file("configs/base_config.yaml"))
viewconfig = rd.yload(rd.load_file("configs/view_config.yaml"))
rd.deep_update(baseconfig, viewconfig)

rawconfig = rd.yload(rd.load_file("configs/mahone_config.yaml"))
config = rd.merge_dicts(baseconfig, rawconfig)
rd.prepare_config(config)

In [ ]:
#print(at.ydump(config))

In [ ]:
import kadlu
from datetime import datetime

import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import seaborn as sns

# mpl.use('module://ipympl.backend_nbagg')
#%matplotlib widget
%matplotlib inline
rd.mpl_set_notebook_params()

show_details = config.settings.show_details

## Read Raw Detection Data & Metadata

In [ ]:
# rd.reload_acoustic()

In [ ]:
detection_df, mdb = rd.read_via_config(config)

In [ ]:
if config.settings.otn_transmitter_patch:
    rd.otn_transmitter_patch_1(mdb)

In [ ]:
df_dets, df_inits, _ = rd.process_intervals(detection_df, mdb)

In [ ]:
detection_df, event_bin_split = rd.detection_rate_grid(df_dets, config.settings.time_bin_length, mdb)
events_df, bins_df = rd.split_by_index(detection_df, event_bin_split)

## Group detection data by (Receiver, Transmitter) pairs

A name is produced for each pairing that reflects their configuration, such as power level, tag family, distance - as determined by parsing the metadata.

In [ ]:
deploy_lat_lon = mdb.deploy.groupby('STATION_NO')[['DEPLOY_LAT','DEPLOY_LONG']].nth(0)
mdb.station_dists_m = rd.calc_station_dists_m(deploy_lat_lon)

if "Receiver/Transmitter" not in mdb.rt_groups.columns:
    rd.add_rt_group_info(events_df, mdb)

In [ ]:
if config.settings.show_details:
    rd.report_station_info(mdb)

In [ ]:
if config.view.show_dr_plots:
    rd.displaymd("# Detection rate plots for data screening")
    for gn, tgroup in bins_df.reset_index().groupby(['Receiver','Transmitter']):
        rd.plot_group_dr(gn, tgroup, mdb)
        plt.show()

## Detection data is merged with environmental variables from kadlu

[Kadlu](https://docs.meridian.cs.dal.ca/kadlu/index.html#) is a Python package which provides functionality for fetching and interpolating environmental data related to ocean ambient nose levels. The `acoustic_tracking` package provides users with the option to integrate environmental data from Kadlu with their own detection datasets. 


To extract environmental data from kadlu, you will need to specify (1) data sources and (2) bounds. Then, using these specifications you can use the `add_kadlu_env_data()` function to automatically extract and interpolate data using the kadlu Python package. 

### Data Sources
A `sources` dictionary is used to specify the variables and data sources you want to retrieve using kadlu. To see a high level overview of available data sources, execute the command below. 
```python
print(kadlu.source_map)
``` 

In [ ]:
print(kadlu.source_map)

In [ ]:
# List Sources
sources = config.data.sources

### Data Boundaries
A `bounds` dictionary is used to specify the spatial and temporal boundaries for which you want to retrieve data. A `north`, `south`, `east`, and `west` value are provided to specify geospatial boundaries, while a `start` and `end` are used to specify temporal boundaries. Optionally, `top` and `bottom` values can be used to limit data to specific depths. 

In [ ]:
# Define Bounds
# Mahone Bay, NS

lat_center = 44.5541333
north_offset = 0.5
south_offset = 2

lon_center = -64.17682
east_offset = 2
west_offset = 2

bounds = dict(start=datetime(2016, 3, 9), end=datetime(2016,3,11),
              south=lat_center - south_offset, west=lon_center - west_offset, 
              north=lat_center + north_offset, east= lon_center + east_offset, 
              top=0, bottom=0)

# OR

bounds = config.bounds

print(bounds)

Sometimes, it can be helpful to have a map visualization of the boundaries you are setting. Here, you can use the `plot_bounds` function to see the boundaries you have specified. Optionally, you can provide receiver locations and metadata to the `plot_bounds` function as well. 

In [ ]:
# Retrieve receiver information
receiver_info = detection_df[['Receiver.lat', 'Receiver.lon', 'Receiver.ID', 
                              'Receiver', 'Receiver.depth']].drop_duplicates()
receiver_locations_df = detection_df[['Receiver.lat', 'Receiver.lon']].drop_duplicates()
receiver_locations = list(zip(receiver_locations_df['Receiver.lat'], receiver_locations_df['Receiver.lon']))

try:
    node_locs = set(list(zip(kadlu_result[1], kadlu_result[2])))
except:
    node_locs = []
    print("Warning: no node locations available. Consider to run this cell again, after the next cell set result approprietly.")

rd.plot_bounds(bounds, receiver_locations=receiver_locations, 
               receiver_info=receiver_info, node_locations=node_locs);

### Add Environmental Variables from kadlu


In [ ]:
df_detections_env, kadlu_result = rd.add_kadlu_env_data(bounds, sources, detection_df)

<a id='custom'></a>
## Reading Custom Data

In addition to integrating environmental data from Kadlu, acoustic_tracking allows you to integrate environmental data contained from custom NetCDF files. This allows users to leverage data from custom datasets which aren't available through Kadlu but might be relevant to the range test of interest. 

Here, we will use the `add_custom_env_data()` function to integrate custom environmental data from HYCOM with our detection dataset. This dataset provides finer resolution data for our region of interest than what is available through the Kadlu interface. 

In [ ]:
# Specify axes to interpolate (the axes which specify the points to interpolate)
axes_to_interpolate = [df_detections_env['Receiver.lat'], 
                       df_detections_env['Receiver.lon'], 
                       [d.timestamp() for d in df_detections_env['datetime']],
                       df_detections_env['Receiver.depth']]

# Specify the variable -> file map. This tells us which data file (net CDF file) contains the data for 
# a specific variable.  
file_map = config.file_map

# Add custom environment data
df_detections_env = rd.environment.add_custom_env_data(axes_to_interpolate, file_map, df_detections_env)

# Look at the new dataset
df_detections_env

### Add tidal information

In [ ]:
if 'tidal' in config.data.keys():
    df_tidal_times = rd.read_ods(config.data.tidal.tidal_times_ods, 1)
    df_tidal_flat = rd.flatten_tidal_table(df_tidal_times, year=config.data.tidal.year)
    df_tidal_interp = rd.tidal_phase(df_tidal_flat, new_times = df_detections_env.datetime)
    df_detections_env = df_detections_env.reset_index().merge(
        df_tidal_interp[["t2","height","dheight_cm_per_hr"]], 
        how="left",
        left_on="datetime", right_index=True).set_index("index")

    if config.view.tidal:
        rd.displaymd("""# Determine tidal heights via interpolation of tidal time tables
In addition to ocean and weather model data, historic tidal tables are available and used here to provide 
additional information about environmental cycles that could be factors of influence on the acoustic data.
""")
        rd.show_tidal_plots(df_tidal_flat, df_tidal_interp,
                            **rd.select_keys(config.data.tidal,
                                             ['tidal_times_output_csv', 'tidal_interpolation_output_csv']))

### Add further calculated columns and prepare for event/bin row split

In [ ]:
for colname in config.data.calculated_columns:
    rd.make_column(df_detections_env, column_name=colname)

In [ ]:
event_bin_dfs = lambda df: rd.split_by_index(df, event_bin_split)
detection_events_df, detection_bins_df = event_bin_dfs(df_detections_env)
rt_group_data = list(df_detections_env.groupby(['Receiver','Transmitter']))

In [ ]:
columns, colnames, column = rd.get_column_info(config)

rd.displaymd("# Plots of detection density and interval lengths <br/> against tidal phase (t2) and {}".format(colnames[column]))

In [ ]:
rd.rcParams.update(config.view.rcParams)
params = rd.make_params(config)

# column is defined above from config
column_name = (column, colnames[column])
skipmsg = False

# each group contains all detections for a particular receiver/transmitter combination
for gn, gr in rt_group_data:
    rt_name = mdb.rt_groups.loc[gn, 'Receiver/Transmitter']
    if len(gr) < params.min_detections:
        if not skipmsg:
            rd.displaymd("**Skipping receiver/transmitter combinations that have insufficient detections:**")
            skipmsg = True
        rd.displaymd("{}".format(rt_name))
        continue

    events_df, bins_df = event_bin_dfs(gr)

    tdfok = bins_df
    if tdfok.empty:
        rd.displaymd("Skipping ")
        rd.displaymd("{}".format(rt_name))
        continue
    rd.process_detections(events_df, params)

    if True:
        rd.plot_with_dr(tdfok, params, column_name=column_name, rt_name=rt_name)

    fig, axs = plt.subplots(nrows=1, ncols=3)
    fig.suptitle(rt_name)

    rd.plot_tidal_phase(tdfok, ax=axs[0])

    rd.plot_with_detection_count(params.out.tdfcount, params.out.tdfmean,
                              params, column_name=column_name,
                              ax=axs[0])    # interval lengths over date range
    if False:
        # comparison in single plot
        #broken: at.plot_with_detection_interval(tdfok, params, column_name=column_name, ax=axs[1])
        ax = params.out.tdfmean.plot("t2","interval", alpha=1, ax=axs[1], c="darkgrey", linewidth=2)
    elif False:
        # old plot type focussing on interval lengths rather than DR
        rd.plot_with_detection_interval_and_rate(events_df, params, column_name=column_name, ax=axs[1])
    else:
        # stacked plot for comparison of quantities
        rd.plot_stack_with_dr(tdfok, params, column_name=column_name, mainax=axs[1])

    rd.plot_per_detection_rate(bins_df, params, column_name=column_name, ax=axs[2])
    #at.plot_per_detection_density(events_df, params, column_name=column_name, ax=axs[2])

    plt.subplots_adjust(wspace=.3)


In [ ]:
from range_driver.plotting import heatmaps

#det_df = detection_events_df
det_df = detection_bins_df

features = det_df[['wavedir', 'waveheight', 'waveperiod', 'salinity_bottom', 'salinity', 
                  'water_v', 'water_v_bottom', 'surf_el', 'water_temp_bottom', 'water_temp', 
                  'water_u_bottom', 'water_u', 't2', 'height', 'dheight_cm_per_hr', 'interval', 
                  'water_vel', 'detection_rate']]

heatmaps.plot_feature_heatmap(features, method='spearman');

## Discussion

In the above visual summary, the **H-N** combinations, i.e. high-power, near distance, are the ones where water velocity shows the least effect on variations in detection rate. This confirms expectations and shows promise for the proposed study method. Next steps include:

- Continue to work with detection rate (DR) as calculated in a fixed grid of time windows
- Compare variations of DR with respect to other environmental variables
- Import other environmental variables automatically via data source APIs (ERDDAP, kadlu.fetch)
- Compare different scenarios of factor importance depending on environmental setting of the study 


## Acknowledgements

The above analysis was performed using [data from OTN](http://members.devel.oceantrack.org/erddap/tabledap/otnunit_aat_detections.html) (provided by Jonathan Pye of OTN), in combination with HYCOM environmental data and tidal data provided by Casey Hilliard (Meridian/Dal), with a synthesized dataset prepared by Matthew Berkowitz (SFU), with project definition and guidance provided by Oliver Kirsebom (Dal) and Ines Hessler (Dal) as part of the [Meridian Network](https://meridian.cs.dal.ca).